# Chunks

## Data

### Set up

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging

logger = logging.getLogger('ML MODEL')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()
# data_zip = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking.tar.gz'
# shutil.unpack_archive(data_zip, '/content/')

In [ ]:
root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking'

In [ ]:
def add_root(s, name):
  temp = os.path.join(data_root, name)
  return str(os.path.join(temp, s))

### AICovidVN

In [ ]:
aicovidvn = os.path.join(data_root, 'AICovidVN/aicovidvn_chunk_metadata.csv')
aicovidvn_df = pd.read_csv(aicovidvn)
aicovidvn_df['chunk_path'] = aicovidvn_df['chunk_path'].apply(add_root, name='AICovidVN')
aicovidvn_df.head()

,uuid,age,gender,label,sr_path,chunk_path,num_chunk
0,3284bcf1-2446-4f3a-ac66-14c76b294177-0,23.0,male,0,aicovidvn_cough_data/3284bcf1-2446-4f3a-ac66-14c76b294177.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/AICovidVN/aicovidvn_cough_chunk/3284bcf1-2446-4f3a-ac66-14c76b294177-0.wav,11
1,3284bcf1-2446-4f3a-ac66-14c76b294177-1,23.0,male,0,aicovidvn_cough_data/3284bcf1-2446-4f3a-ac66-14c76b294177.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/AICovidVN/aicovidvn_cough_chunk/3284bcf1-2446-4f3a-ac66-14c76b294177-1.wav,11
2,3284bcf1-2446-4f3a-ac66-14c76b294177-2,23.0,male,0,aicovidvn_cough_data/3284bcf1-2446-4f3a-ac66-14c76b294177.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/AICovidVN/aicovidvn_cough_chunk/3284bcf1-2446-4f3a-ac66-14c76b294177-2.wav,11
3,3284bcf1-2446-4f3a-ac66-14c76b294177-3,23.0,male,0,aicovidvn_cough_data/3284bcf1-2446-4f3a-ac66-14c76b294177.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/AICovidVN/aicovidvn_cough_chunk/3284bcf1-2446-4f3a-ac66-14c76b294177-3.wav,11
4,3284bcf1-2446-4f3a-ac66-14c76b294177-4,23.0,male,0,aicovidvn_cough_data/3284bcf1-2446-4f3a-ac66-14c76b294177.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/AICovidVN/aicovidvn_cough_chunk/3284bcf1-2446-4f3a-ac66-14c76b294177-4.wav,11


### Coswara

In [ ]:
coswara = os.path.join(data_root, 'coswara/coswara_chunk_metadata.csv')
coswara_df = pd.read_csv(coswara)
coswara_df['chunk_path'] = coswara_df['chunk_path'].apply(add_root, name='coswara')
coswara_df.head()

,uuid,age,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,is_smoker,is_cold,is_hypertension,is_diabetes,is_cough,date_of_ct_scan,has_ctScan,ct_score,is_diarrheoa,is_fever,is_loss_of_smell,is_muscle_pain,test_type,test_date,test_status,is_using_mask,vaccination_status,is_breathing_difficulty,is_others_resp,is_fatigue,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,label,sr_cough,chunk_path,num_chunk
0,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coswara/coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0.wav,8
1,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coswara/coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1.wav,8
2,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coswara/coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2.wav,8
3,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coswara/coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3.wav,8
4,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coswara/coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4.wav,8


### Coughvid

In [ ]:
coughvid = os.path.join(data_root, 'coughvid/coughvid_chunk_metadata.csv')
coughvid_df = pd.read_csv(coughvid)
coughvid_df['chunk_path'] = coughvid_df['chunk_path'].apply(add_root, name='coughvid')
coughvid_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,quality_1,cough_type_1,dyspnea_1,wheezing_1,stridor_1,choking_1,congestion_1,nothing_1,diagnosis_1,severity_1,quality_2,cough_type_2,dyspnea_2,wheezing_2,stridor_2,choking_2,congestion_2,nothing_2,diagnosis_2,severity_2,quality_3,dyspnea_3,wheezing_3,stridor_3,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,label,sr_cough,chunk_path,num_chunk
0,00039425-7f3a-42aa-ac13-834aaa2b6b92-0,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coughvid/coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-0.wav,3
1,00039425-7f3a-42aa-ac13-834aaa2b6b92-1,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coughvid/coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-1.wav,3
2,00039425-7f3a-42aa-ac13-834aaa2b6b92-2,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coughvid/coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-2.wav,3
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coughvid/coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0.wav,3
4,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coughvid/coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1.wav,3


## Feature extraction

### Set up

In [ ]:
import librosa
import librosa.display as ldp
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
# clear_output()

In [ ]:
START_SAMPLE = 10000
SAMPLES_TO_CONSIDER = 22050


def show_feature(feature, sample_rate):
  plt.figure(figsize=(15, 10))
  ldp.specshow(feature,
               x_axis="time",
               y_axis="mel", 
               sr=sample_rate)
  plt.colorbar(format="%+2.f")
  plt.show()


def lengthen_cough(data: np.array, n):
  temp = data
  for i in range(n-1):
    temp = np.concatenate([temp, data])
  return temp

def mel_spectrogram(path: str, nfft=2048, hoplen=512, nmels=224):
  result=np.array([])
  data, sample_rate = librosa.load(path)
  data = data[START_SAMPLE: SAMPLES_TO_CONSIDER]
  data = lengthen_cough(data, 10)
  mel_spec = librosa.feature.melspectrogram(y=data,
                                            sr=sample_rate,
                                            n_fft=nfft,
                                            hop_length=hoplen,
                                            n_mels=nmels)
  # logger.info(path)
  log_mel_spec = librosa.power_to_db(mel_spec) # log mel
  log_mean = np.mean(log_mel_spec.T, axis=0) # take mean
  result=np.hstack((result, log_mean))
  return result

def n_mfcc(path: str, nfft=2048, hoplen=512, nmels=224, nmfcc=13):
  result=np.array([])
  data, sample_rate = librosa.load(path)
  data = data[START_SAMPLE: SAMPLES_TO_CONSIDER]
  data = lengthen_cough(data, 10)
  mfcc = librosa.feature.mfcc(y=data,
                              sr=sample_rate,
                              n_fft=nfft,
                              hop_length=hoplen,
                              n_mfcc=nmfcc)
  # logger.info(path)
  mfcc_mean = np.mean(mfcc.T, axis=0) # take mean
  result=np.hstack((result, mfcc_mean))
  return result

In [ ]:
p = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/AICovidVN/aicovidvn_cough_chunk/3284bcf1-2446-4f3a-ac66-14c76b294177-0.wav'

In [ ]:
# num_mfcc=13
# def extract_feature(path, mfcc, chroma, mel, tempo):
#   data, sample_rate = librosa.load(path)
  
#   if chroma:
#     stft = np.abs(librosa.stft(data))

#   result=np.array([])
#   if mfcc:
#     mfccs=np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=num_mfcc).T, axis=0)
#     result=np.hstack((result, mfccs))
#   if chroma:
#     chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
#     result=np.hstack((result, chroma))
#   if mel:
#     mel=np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
#     result=np.hstack((result, mel))
#   if tempo:
#     tempo=np.mean(librosa.feature.tempogram(y=data, sr=sample_rate).T, axis=0)
#     result=np.hstack((result, tempo))

#   return result

In [ ]:
def extract_feature(data: pd.DataFrame, f_type: str, nmfcc=13):
  features = []
  if f_type.lower() == 'melspectrogram':
    for fp in data['chunk_path'].tolist():
      features.append(mel_spectrogram(path=fp))
      if len(features) % 1000 == 0:
        logger.info(f'{len(features)} chunks extracted !!!')

  if f_type.lower() == 'mfcc':
    for fp in data['chunk_path'].tolist():
      features.append(n_mfcc(path=fp, nmfcc=nmfcc))
      if len(features) % 1000 == 0:
        logger.info(f'{len(features)} chunks extracted !!!')    
  logger.info(f'{len(features)} chunks extracted !!!')
  logger.info('Feature extraction: Done !!!')
  return np.array(features)

### AICovidVN

#### Mel-spectrogram

In [ ]:
features = extract_feature(aicovidvn_df, 'melspectrogram')

In [ ]:
mel_df = pd.DataFrame(features)
label = aicovidvn_df['label'].tolist()
mel_df['label'] = label


In [ ]:
mel_df['label'].value_counts()

0    26027
1     5515
Name: label, dtype: int64

In [ ]:
mel_df.to_csv(os.path.join(root, 'melspectrogram_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'melspectrogram_aicovidvn_features.csv'))

2022-02-17 03:55:56,787 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/melspectrogram_aicovidvn_features.csv


#### MFCC

In [ ]:
del features
features = extract_feature(aicovidvn_df, 'mfcc')

In [ ]:
mfcc_df = pd.DataFrame(features)
label = aicovidvn_df['label'].tolist()
mfcc_df['label'] = label

In [ ]:
mfcc_df['label'].value_counts()

0    26027
1     5515
Name: label, dtype: int64

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_13_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_13_aicovidvn_features.csv'))

2022-02-17 04:49:24,928 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_13_aicovidvn_features.csv


In [ ]:
del features
features = extract_feature(aicovidvn_df, 'mfcc', nmfcc=26)

In [ ]:
mfcc_df = pd.DataFrame(features)
label = aicovidvn_df['label'].tolist()
mfcc_df['label'] = label
mfcc_df.to_csv(os.path.join(root, 'mfcc_26_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_26_aicovidvn_features.csv'))

2022-02-17 05:36:08,344 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_26_aicovidvn_features.csv


In [ ]:
del features
features = extract_feature(aicovidvn_df, 'mfcc', nmfcc=39)

In [ ]:
mfcc_df = pd.DataFrame(features)
label = aicovidvn_df['label'].tolist()
mfcc_df['label'] = label
mfcc_df.to_csv(os.path.join(root, 'mfcc_39_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_39_aicovidvn_features.csv'))

2022-02-17 06:57:06,318 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_39_aicovidvn_features.csv


### Coswara

#### Mel-spectrogram

In [ ]:
features = extract_feature(coswara_df, 'melspectrogram')

2022-02-17 08:30:33,809 - ML MODEL - INFO - 1000 chunks extracted !!!
2022-02-17 08:39:11,362 - ML MODEL - INFO - 2000 chunks extracted !!!
2022-02-17 08:52:21,600 - ML MODEL - INFO - 3000 chunks extracted !!!
2022-02-17 09:05:23,568 - ML MODEL - INFO - 4000 chunks extracted !!!
2022-02-17 09:18:28,472 - ML MODEL - INFO - 5000 chunks extracted !!!
2022-02-17 09:31:23,510 - ML MODEL - INFO - 6000 chunks extracted !!!
2022-02-17 09:44:20,988 - ML MODEL - INFO - 7000 chunks extracted !!!
2022-02-17 09:57:08,811 - ML MODEL - INFO - 8000 chunks extracted !!!
2022-02-17 10:10:05,941 - ML MODEL - INFO - 9000 chunks extracted !!!
2022-02-17 10:22:57,243 - ML MODEL - INFO - 10000 chunks extracted !!!
2022-02-17 10:35:27,565 - ML MODEL - INFO - 11000 chunks extracted !!!
2022-02-17 10:48:12,942 - ML MODEL - INFO - 12000 chunks extracted !!!
2022-02-17 11:01:08,439 - ML MODEL - INFO - 13000 chunks extracted !!!
2022-02-17 11:14:13,120 - ML MODEL - INFO - 14000 chunks extracted !!!
2022-02-17 11:2

In [ ]:
mel_df = pd.DataFrame(features)
label = coswara_df['label'].tolist()
mel_df['label'] = label

In [ ]:
mel_df.to_csv(os.path.join(root, 'melspectrogram_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'melspectrogram_coswara_features.csv'))

2022-02-17 13:08:57,999 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/melspectrogram_coswara_features.csv


#### MFCC

In [ ]:
del features
features = extract_feature(coswara_df, 'mfcc')

2022-02-17 13:11:17,257 - ML MODEL - INFO - 1000 chunks extracted !!!
2022-02-17 13:13:20,389 - ML MODEL - INFO - 2000 chunks extracted !!!
2022-02-17 13:15:24,985 - ML MODEL - INFO - 3000 chunks extracted !!!
2022-02-17 13:17:26,365 - ML MODEL - INFO - 4000 chunks extracted !!!
2022-02-17 13:19:27,036 - ML MODEL - INFO - 5000 chunks extracted !!!
2022-02-17 13:21:28,109 - ML MODEL - INFO - 6000 chunks extracted !!!
2022-02-17 13:23:31,410 - ML MODEL - INFO - 7000 chunks extracted !!!
2022-02-17 13:25:30,723 - ML MODEL - INFO - 8000 chunks extracted !!!
2022-02-17 13:27:31,371 - ML MODEL - INFO - 9000 chunks extracted !!!
2022-02-17 13:29:32,142 - ML MODEL - INFO - 10000 chunks extracted !!!
2022-02-17 13:31:33,404 - ML MODEL - INFO - 11000 chunks extracted !!!
2022-02-17 13:33:38,175 - ML MODEL - INFO - 12000 chunks extracted !!!
2022-02-17 13:35:42,502 - ML MODEL - INFO - 13000 chunks extracted !!!
2022-02-17 13:37:48,346 - ML MODEL - INFO - 14000 chunks extracted !!!
2022-02-17 13:3

In [ ]:
mfcc_df = pd.DataFrame(features)
label = coswara_df['label'].tolist()
mfcc_df['label'] = label

In [ ]:
mfcc_df['label'].value_counts()

healthy                        14348
positive_mild                   2472
no_resp_illness_exposed         1861
resp_illness_not_identified     1595
positive_moderate               1091
recovered_full                   928
positive_asymp                   583
Name: label, dtype: int64

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_13_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_13_coswara_features.csv'))

2022-02-17 13:56:18,011 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_13_coswara_features.csv


In [ ]:
del features
features = extract_feature(coswara_df, 'mfcc', nmfcc=26)

2022-02-17 13:58:17,578 - ML MODEL - INFO - 1000 chunks extracted !!!
2022-02-17 14:00:19,365 - ML MODEL - INFO - 2000 chunks extracted !!!
2022-02-17 14:02:24,863 - ML MODEL - INFO - 3000 chunks extracted !!!
2022-02-17 14:04:29,183 - ML MODEL - INFO - 4000 chunks extracted !!!
2022-02-17 14:06:30,028 - ML MODEL - INFO - 5000 chunks extracted !!!
2022-02-17 14:08:30,996 - ML MODEL - INFO - 6000 chunks extracted !!!
2022-02-17 14:10:32,610 - ML MODEL - INFO - 7000 chunks extracted !!!
2022-02-17 14:12:32,890 - ML MODEL - INFO - 8000 chunks extracted !!!
2022-02-17 14:14:36,710 - ML MODEL - INFO - 9000 chunks extracted !!!
2022-02-17 14:16:37,179 - ML MODEL - INFO - 10000 chunks extracted !!!
2022-02-17 14:18:38,222 - ML MODEL - INFO - 11000 chunks extracted !!!
2022-02-17 14:20:41,569 - ML MODEL - INFO - 12000 chunks extracted !!!
2022-02-17 14:22:47,057 - ML MODEL - INFO - 13000 chunks extracted !!!
2022-02-17 14:24:57,085 - ML MODEL - INFO - 14000 chunks extracted !!!
2022-02-17 14:2

In [ ]:
mfcc_df = pd.DataFrame(features)
label = coswara_df['label'].tolist()
mfcc_df['label'] = label
mfcc_df.to_csv(os.path.join(root, 'mfcc_26_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_26_coswara_features.csv'))

2022-02-17 14:43:31,281 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_26_coswara_features.csv


In [ ]:
del features
features = extract_feature(coswara_df, 'mfcc', nmfcc=39)

2022-02-17 14:45:34,477 - ML MODEL - INFO - 1000 chunks extracted !!!
2022-02-17 14:47:34,202 - ML MODEL - INFO - 2000 chunks extracted !!!
2022-02-17 14:49:37,381 - ML MODEL - INFO - 3000 chunks extracted !!!
2022-02-17 14:51:37,726 - ML MODEL - INFO - 4000 chunks extracted !!!
2022-02-17 14:53:38,395 - ML MODEL - INFO - 5000 chunks extracted !!!
2022-02-17 14:55:43,783 - ML MODEL - INFO - 6000 chunks extracted !!!
2022-02-17 14:57:46,425 - ML MODEL - INFO - 7000 chunks extracted !!!
2022-02-17 14:59:47,585 - ML MODEL - INFO - 8000 chunks extracted !!!
2022-02-17 15:01:53,752 - ML MODEL - INFO - 9000 chunks extracted !!!
2022-02-17 15:03:58,978 - ML MODEL - INFO - 10000 chunks extracted !!!
2022-02-17 15:06:08,641 - ML MODEL - INFO - 11000 chunks extracted !!!
2022-02-17 15:08:12,498 - ML MODEL - INFO - 12000 chunks extracted !!!
2022-02-17 15:10:18,317 - ML MODEL - INFO - 13000 chunks extracted !!!
2022-02-17 15:12:26,110 - ML MODEL - INFO - 14000 chunks extracted !!!
2022-02-17 15:1

In [ ]:
mfcc_df = pd.DataFrame(features)
label = coswara_df['label'].tolist()
mfcc_df['label'] = label
mfcc_df.to_csv(os.path.join(root, 'mfcc_39_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_39_coswara_features.csv'))

2022-02-17 15:31:00,905 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_39_coswara_features.csv


### Coughvid

#### Melspectrogram

In [ ]:
features = extract_feature(coughvid_df, 'melspectrogram')

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
2022-02-21 13:12:01,807 - ML MODEL - INFO - 1000 chunks extracted !!!
2022-02-21 13:16:45,404 - ML MODEL - INFO - 2000 chunks extracted !!!
2022-02-21 13:21:28,500 - ML MODEL - INFO - 3000 chunks extracted !!!
2022-02-21 13:26:09,156 - ML MODEL - INFO - 4000 chunks extracted !!!
2022-02-21 13:30:57,061 - ML MODEL - INFO - 5000 chunks extracted !!!
2022-02-21 13:35:57,050 - ML MODEL - INFO - 6000 chunks extracted !!!
2022-02-21 13:40:40,987 - ML MODEL - INFO - 7000 chunks extracted !!!
2022-02-21 13:45:25,582 - ML MODEL - INFO - 8000 chunks extracted !!!
2022-02-21 13:50:12,345 - ML MODEL - INFO - 9000 chunks extracted !!!
2022-02-21 13:54:56,114 - ML MODEL - INFO - 10000 chunks extracted !!!
2022-02-21 13:59:46,049 - ML MODEL - INFO - 11000 chunks extracted !!!
2022-02-21 14:04:35,370 - ML MODEL - I

In [ ]:
mel_df = pd.DataFrame(features)
label = coughvid_df['label'].tolist()
mel_df['label'] = label

In [ ]:
mel_df.to_csv(os.path.join(root, 'melspectrogram_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'melspectrogram_coughvid_features.csv'))

2022-02-21 19:57:51,794 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/melspectrogram_coughvid_features.csv


#### MFCC

In [ ]:
# del features
features1 = extract_feature(coughvid_df[:40000], 'mfcc')
mfcc_df_1 = pd.DataFrame(features1)
label_1 = coughvid_df['label'][:40000].tolist()
mfcc_df_1['label'] = label_1
mfcc_df_1.to_csv(os.path.join(root, '1mfcc_13_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, '1mfcc_13_coughvid_features.csv'))

2022-02-23 05:25:37,859 - ML MODEL - INFO - 1000 chunks extracted !!!
2022-02-23 05:27:48,288 - ML MODEL - INFO - 2000 chunks extracted !!!
2022-02-23 05:29:54,752 - ML MODEL - INFO - 3000 chunks extracted !!!
2022-02-23 05:32:06,041 - ML MODEL - INFO - 4000 chunks extracted !!!
2022-02-23 05:34:13,731 - ML MODEL - INFO - 5000 chunks extracted !!!
2022-02-23 05:36:21,643 - ML MODEL - INFO - 6000 chunks extracted !!!
2022-02-23 05:38:28,532 - ML MODEL - INFO - 7000 chunks extracted !!!
2022-02-23 05:40:35,314 - ML MODEL - INFO - 8000 chunks extracted !!!
2022-02-23 05:42:46,830 - ML MODEL - INFO - 9000 chunks extracted !!!
2022-02-23 05:44:54,703 - ML MODEL - INFO - 10000 chunks extracted !!!
2022-02-23 05:47:00,748 - ML MODEL - INFO - 11000 chunks extracted !!!
2022-02-23 05:49:08,410 - ML MODEL - INFO - 12000 chunks extracted !!!
2022-02-23 05:51:15,447 - ML MODEL - INFO - 13000 chunks extracted !!!
2022-02-23 05:53:24,269 - ML MODEL - INFO - 14000 chunks extracted !!!
2022-02-23 05:5

In [ ]:
# del features
features2 = extract_feature(coughvid_df[40000:], 'mfcc')
mfcc_df_2 = pd.DataFrame(features2)
label_2 = coughvid_df['label'][40000:].tolist()
mfcc_df_2['label'] = label_2
mfcc_df_2.to_csv(os.path.join(root, '2mfcc_13_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, '2mfcc_13_coughvid_features.csv'))

2022-02-26 07:11:15,728 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/2mfcc_13_coughvid_features.csv


In [ ]:
mfcc_df_1 = pd.read_csv(os.path.join(root, '1mfcc_13_coughvid_features.csv'))
mfcc_df_2 = pd.read_csv(os.path.join(root, '2mfcc_13_coughvid_features.csv'))

In [ ]:
mfcc_df = pd.concat([mfcc_df_1, mfcc_df_2], ignore_index=True)
mfcc_df.to_csv(os.path.join(root, 'mfcc_13_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_13_coughvid_features.csv'))

2022-02-26 15:00:12,230 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_13_coughvid_features.csv


In [ ]:
mfcc_df.shape

(84981, 14)

In [ ]:
del features
features = extract_feature(coughvid_df, 'mfcc', nmfcc=26)

In [ ]:
mfcc_df = pd.DataFrame(features)
label = coughvid_df['label'].tolist()
mfcc_df['label'] = label
mfcc_df.to_csv(os.path.join(root, 'mfcc_26_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_26_coughvid_features.csv'))

In [ ]:
mfcc_df_1 = pd.read_csv(os.path.join(root, '1mfcc_26_coughvid_features.csv'))
mfcc_df_2 = pd.read_csv(os.path.join(root, '2mfcc_26_coughvid_features.csv'))

In [ ]:
mfcc_df = pd.concat([mfcc_df_1, mfcc_df_2], ignore_index=True)
mfcc_df.to_csv(os.path.join(root, 'mfcc_26_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_26_coughvid_features.csv'))
mfcc_df.shape

2022-02-26 15:03:32,811 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_26_coughvid_features.csv


(84981, 27)

In [ ]:
del features
features = extract_feature(coughvid_df, 'mfcc', nmfcc=39)

In [ ]:
mfcc_df = pd.DataFrame(features)
label = coughvid_df['label'].tolist()
mfcc_df['label'] = label
mfcc_df.to_csv(os.path.join(root, 'mfcc_39_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_39_coughvid_features.csv'))

In [ ]:
mfcc_df_1 = pd.read_csv(os.path.join(root, '1mfcc_39_coughvid_features.csv'))
mfcc_df_2 = pd.read_csv(os.path.join(root, '2mfcc_39_coughvid_features.csv'))

In [ ]:
mfcc_df = pd.concat([mfcc_df_1, mfcc_df_2], ignore_index=True)
mfcc_df.to_csv(os.path.join(root, 'mfcc_39_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_39_coughvid_features.csv'))
mfcc_df.shape

2022-02-26 15:04:26,953 - ML MODEL - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/mfcc_39_coughvid_features.csv


(84981, 40)

In [ ]:
# Audio('/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking/coughvid/coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-0.wav')

# Full audio
* Extract features for cough audio:
  * AICovidVN: 5,247
  * Coswara: 4,465
  * Coughvid: 20,072

## Data

### Set up

In [1]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings
warnings.filterwarnings("ignore")

logger = logging.getLogger('FULL AUDIO 1-D')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()
# data_zip = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing/cough_chunking.tar.gz'
# shutil.unpack_archive(data_zip, '/content/')

In [2]:
root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data'

In [3]:
def add_root(s, name):
  temp = os.path.join(data_root, name)
  return str(os.path.join(temp, s))

### AICovidVN

In [7]:
aicovidvn = os.path.join(data_root, 'AICovidVN/aicovidvn_metadata.csv')
aicovidvn_df = pd.read_csv(aicovidvn)
aicovidvn_df['cough_path'] = aicovidvn_df['cough_path'].apply(add_root, name='AICovidVN')
aicovidvn_df['label'] = aicovidvn_df['assessment_result']
aicovidvn_df.head()

,uuid,subject_gender,subject_age,assessment_result,cough_path,label
0,3284bcf1-2446-4f3a-ac66-14c76b294177,male,23.0,0,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/AICovidVN/aicovidvn_cough_data/3284bcf1-2446-4f3a-ac66-14c76b294177.wav,0
1,431334e1-5946-4576-bb51-8e342ccc22b4,NaN,NaN,0,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/AICovidVN/aicovidvn_cough_data/431334e1-5946-4576-bb51-8e342ccc22b4.wav,0
2,1d6fac4b-1e7f-4bdc-81cd-3a720bfbb1e1,NaN,NaN,0,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/AICovidVN/aicovidvn_cough_data/1d6fac4b-1e7f-4bdc-81cd-3a720bfbb1e1.wav,0
3,c7ee0695-b2e7-4beb-b904-f1455c9609d9,male,49.0,0,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/AICovidVN/aicovidvn_cough_data/c7ee0695-b2e7-4beb-b904-f1455c9609d9.wav,0
4,dd541704-b696-4181-8fd8-816daac0fcf9,NaN,NaN,0,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/AICovidVN/aicovidvn_cough_data/dd541704-b696-4181-8fd8-816daac0fcf9.wav,0


In [8]:
Audio(aicovidvn_df.loc[0, 'cough_path'])

### Coswara

In [ ]:
coswara = os.path.join(data_root, 'coswara/coswara_metadata.csv')
coswara_df = pd.read_csv(coswara)
coswara_df['cough_path'] = coswara_df['cough_path'].apply(add_root, name='coswara')
coswara_df['label'] = coswara_df['covid_status']
coswara_df.head()

,id,age,covid_status,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,...,is_others_resp,is_fatigue,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,cough_path,label
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,healthy
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/AxuYWBN0jFVLINCBqIW5aZmGCdu1/cough-shallow.wav,healthy
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/C5eIsssb9GSkaAgIfsHMHeR6fSh1/cough-shallow.wav,healthy
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/YjbEAECMBIaZKyfqOvWy5DDImUb2/cough-shallow.wav,healthy
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/aGOvk4ji0cVqIzCs1jHnzlw2UEy2/cough-shallow.wav,healthy


In [ ]:
Audio(coswara_df.loc[0, 'cough_path'])

### Coughvid

In [ ]:
coughvid = os.path.join(data_root, 'coughvid/coughvid_metadata.csv')
coughvid_df = pd.read_csv(coughvid)
coughvid_df['cough_path'] = coughvid_df['cough_path'].apply(add_root, name='coughvid')
coughvid_df['label'] = coughvid_df['status']
coughvid_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,...,wheezing_3,stridor_3,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,cough_path,label
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,healthy
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,healthy
2,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0012c608-33d0-4ef7-bde3-75a0b1a0024e.webm,NaN
3,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,NaN,NaN,21.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm,healthy
4,001c85a8-cc4d-4921-9297-848be52d4715,2020-04-17T15:24:35.822355+00:00,0.0735,40.6,-3.6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001c85a8-cc4d-4921-9297-848be52d4715.webm,NaN


##Feature extraction

### Set up

In [9]:
import librosa
import librosa.display as ldp
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
# clear_output()

In [10]:
START_SAMPLE = 10000
SAMPLES_TO_CONSIDER = 135440 # 5 seconds


def lengthen_cough(data: np.array, n):
  temp = data
  for i in range(n-1):
    temp = np.concatenate([temp, data])
  return temp

def mel_spectrogram(path: str, nfft=2048, hoplen=512, nmels=224):
  result=np.array([])
  data, sample_rate = librosa.load(path)
  data = librosa.util.fix_length(data, SAMPLES_TO_CONSIDER) # take 5 seconds
  data = data[START_SAMPLE:]
  # data = lengthen_cough(data, 10)
  mel_spec = librosa.feature.melspectrogram(y=data,
                                            sr=sample_rate,
                                            n_fft=nfft,
                                            hop_length=hoplen,
                                            n_mels=nmels)
  # logger.info(path)
  log_mel_spec = librosa.power_to_db(mel_spec) # log mel
  log_mean = np.mean(log_mel_spec.T, axis=0) # take mean
  result=np.hstack((result, log_mean))
  return result

def n_mfcc(path: str, nfft=2048, hoplen=512, nmels=224, nmfcc=13):
  result=np.array([])
  data, sample_rate = librosa.load(path)
  data = librosa.util.fix_length(data, SAMPLES_TO_CONSIDER) # take 5 seconds
  data = data[START_SAMPLE:]
  # data = lengthen_cough(data, 10)
  mfcc = librosa.feature.mfcc(y=data,
                              sr=sample_rate,
                              n_fft=nfft,
                              hop_length=hoplen,
                              n_mfcc=nmfcc)
  # logger.info(path)
  mfcc_mean = np.mean(mfcc.T, axis=0) # take mean
  result=np.hstack((result, mfcc_mean))
  return result

In [11]:
def extract_feature(data: pd.DataFrame, f_type: str, nmfcc=13):
  features = []
  labels = []

  if f_type.lower() == 'melspectrogram':
    for fp, lb in zip(data['cough_path'].tolist(), data['label'].tolist()):
      try:
        features.append(mel_spectrogram(path=fp))
        labels.append(lb)
        if len(features) % 1000 == 0:
          logger.info(f'{len(features)} audios extracted !!!')
      except:
        logger.info(f'ERROR COUGH: {fp}')

  if f_type.lower() == 'mfcc':
    for fp, lb in zip(data['cough_path'].tolist(), data['label'].tolist()):
      try:
        features.append(n_mfcc(path=fp, nmfcc=nmfcc))
        labels.append(lb)
        if len(features) % 1000 == 0:
          logger.info(f'{len(features)} audios extracted !!!')  
      except:
          logger.info(f'ERROR COUGH: {fp}')

  logger.info(f'{len(features)} audios extracted !!!')
  logger.info('Feature extraction: Done !!!')
  return np.array(features), labels

### AICovidVN

#### Mel-spectrogram

In [16]:
features, labels = extract_feature(aicovidvn_df, 'melspectrogram')
mel_df = pd.DataFrame(features)
mel_df['label'] = labels

2022-03-07 11:42:44,633 - FULL AUDIO 1-D - INFO - 1000 audios extracted !!!
2022-03-07 11:51:41,396 - FULL AUDIO 1-D - INFO - 2000 audios extracted !!!
2022-03-07 12:01:11,196 - FULL AUDIO 1-D - INFO - 3000 audios extracted !!!
2022-03-07 12:10:37,811 - FULL AUDIO 1-D - INFO - 4000 audios extracted !!!
2022-03-07 12:19:49,371 - FULL AUDIO 1-D - INFO - 5000 audios extracted !!!
2022-03-07 12:22:08,551 - FULL AUDIO 1-D - INFO - 5247 audios extracted !!!
2022-03-07 12:22:08,559 - FULL AUDIO 1-D - INFO - Feature extraction: Done !!!


In [17]:
mel_df.to_csv(os.path.join(root, 'melspectrogram_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'melspectrogram_aicovidvn_features.csv'))

2022-03-07 12:22:10,627 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/melspectrogram_aicovidvn_features.csv


#### MFCC

In [ ]:
# del features
features, labels = extract_feature(aicovidvn_df, 'mfcc')
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_13_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_13_aicovidvn_features.csv'))

In [12]:
# del features
features, labels = extract_feature(aicovidvn_df, 'mfcc', nmfcc=26)
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

2022-03-07 13:30:12,844 - FULL AUDIO 1-D - INFO - 1000 audios extracted !!!
2022-03-07 13:39:48,934 - FULL AUDIO 1-D - INFO - 2000 audios extracted !!!
2022-03-07 13:49:59,598 - FULL AUDIO 1-D - INFO - 3000 audios extracted !!!
2022-03-07 13:59:59,017 - FULL AUDIO 1-D - INFO - 4000 audios extracted !!!
2022-03-07 14:09:58,489 - FULL AUDIO 1-D - INFO - 5000 audios extracted !!!
2022-03-07 14:12:27,788 - FULL AUDIO 1-D - INFO - 5247 audios extracted !!!
2022-03-07 14:12:27,789 - FULL AUDIO 1-D - INFO - Feature extraction: Done !!!


In [13]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_26_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_26_aicovidvn_features.csv'))

2022-03-07 14:12:28,974 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/mfcc_26_aicovidvn_features.csv


In [ ]:
del features
features, labels = extract_feature(aicovidvn_df, 'mfcc', nmfcc=39)
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_39_aicovidvn_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_39_aicovidvn_features.csv'))

### Coswara

#### Mel-spectrogram

In [ ]:
features, labels = extract_feature(coswara_df, 'melspectrogram')
mel_df = pd.DataFrame(features)
mel_df['label'] = labels

2022-03-06 10:31:27,354 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20210816/WcRpZenx0WWQmzOUPTHEEGlyJAg1/cough-shallow.wav
2022-03-06 10:32:09,566 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200419/XFX3DxpzWlTsqde0wmliVzvRXnf1/cough-shallow.wav
2022-03-06 10:32:31,769 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200824/pBBuvcoBj7hjmNVYFICT4hQYRGw1/cough-shallow.wav
2022-03-06 10:33:02,775 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200820/Tfvspm3rapd3ZLyAoMG36VxNQnr2/cough-shallow.wav
2022-03-06 10:33:05,677 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswa

In [ ]:
mel_df.to_csv(os.path.join(root, 'melspectrogram_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'melspectrogram_coswara_features.csv'))

2022-03-06 11:13:38,801 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/melspectrogram_coswara_features.csv


#### MFCC

In [ ]:
# del features
features, labels = extract_feature(coswara_df, 'mfcc')
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

2022-03-06 11:22:01,147 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20210816/WcRpZenx0WWQmzOUPTHEEGlyJAg1/cough-shallow.wav
2022-03-06 11:22:20,866 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200419/XFX3DxpzWlTsqde0wmliVzvRXnf1/cough-shallow.wav
2022-03-06 11:22:31,649 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200824/pBBuvcoBj7hjmNVYFICT4hQYRGw1/cough-shallow.wav
2022-03-06 11:22:44,895 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200820/Tfvspm3rapd3ZLyAoMG36VxNQnr2/cough-shallow.wav
2022-03-06 11:22:46,105 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswa

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_13_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_13_coswara_features.csv'))

2022-03-06 11:45:06,189 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/mfcc_13_coswara_features.csv


In [ ]:
del features
features, labels = extract_feature(coswara_df, 'mfcc', nmfcc=26)
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

2022-03-06 11:45:17,473 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20210816/WcRpZenx0WWQmzOUPTHEEGlyJAg1/cough-shallow.wav
2022-03-06 11:45:37,176 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200419/XFX3DxpzWlTsqde0wmliVzvRXnf1/cough-shallow.wav
2022-03-06 11:45:47,908 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200824/pBBuvcoBj7hjmNVYFICT4hQYRGw1/cough-shallow.wav
2022-03-06 11:46:01,132 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200820/Tfvspm3rapd3ZLyAoMG36VxNQnr2/cough-shallow.wav
2022-03-06 11:46:02,331 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswa

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_26_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_26_coswara_features.csv'))

2022-03-06 12:08:20,643 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/mfcc_26_coswara_features.csv


In [ ]:
del features
features, labels = extract_feature(coswara_df, 'mfcc', nmfcc=39)
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

2022-03-06 12:08:31,873 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20210816/WcRpZenx0WWQmzOUPTHEEGlyJAg1/cough-shallow.wav
2022-03-06 12:08:51,534 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200419/XFX3DxpzWlTsqde0wmliVzvRXnf1/cough-shallow.wav
2022-03-06 12:09:02,261 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200824/pBBuvcoBj7hjmNVYFICT4hQYRGw1/cough-shallow.wav
2022-03-06 12:09:15,358 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200820/Tfvspm3rapd3ZLyAoMG36VxNQnr2/cough-shallow.wav
2022-03-06 12:09:16,560 - FULL AUDIO 1-D - INFO - ERROR COUGH: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswa

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_39_coswara_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_39_coswara_features.csv'))

2022-03-06 12:31:30,151 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/mfcc_39_coswara_features.csv


### Coughvid

#### Mel-spectrogram

In [ ]:
features, labels = extract_feature(coughvid_df, 'melspectrogram')
mel_df = pd.DataFrame(features)
mel_df['label'] = labels

In [ ]:
mel_df.to_csv(os.path.join(root, 'melspectrogram_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'melspectrogram_coughvid_features.csv'))

#### MFCC

In [ ]:
features, labels = extract_feature(coughvid_df, 'mfcc')
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

2022-03-06 23:51:42,095 - FULL AUDIO 1-D - INFO - 1000 audios extracted !!!
2022-03-07 00:06:09,244 - FULL AUDIO 1-D - INFO - 2000 audios extracted !!!
2022-03-07 00:20:39,592 - FULL AUDIO 1-D - INFO - 3000 audios extracted !!!
2022-03-07 00:35:06,918 - FULL AUDIO 1-D - INFO - 4000 audios extracted !!!
2022-03-07 00:49:49,148 - FULL AUDIO 1-D - INFO - 5000 audios extracted !!!
2022-03-07 01:04:18,581 - FULL AUDIO 1-D - INFO - 6000 audios extracted !!!
2022-03-07 01:18:55,317 - FULL AUDIO 1-D - INFO - 7000 audios extracted !!!
2022-03-07 01:33:31,177 - FULL AUDIO 1-D - INFO - 8000 audios extracted !!!
2022-03-07 01:47:52,612 - FULL AUDIO 1-D - INFO - 9000 audios extracted !!!
2022-03-07 02:02:38,285 - FULL AUDIO 1-D - INFO - 10000 audios extracted !!!
2022-03-07 02:17:41,247 - FULL AUDIO 1-D - INFO - 11000 audios extracted !!!
2022-03-07 02:32:47,056 - FULL AUDIO 1-D - INFO - 12000 audios extracted !!!
2022-03-07 02:47:53,142 - FULL AUDIO 1-D - INFO - 13000 audios extracted !!!
2022-03-

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_13_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_13_coughvid_features.csv'))

2022-03-07 04:34:33,908 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/mfcc_13_coughvid_features.csv


In [ ]:
# del features
features, labels = extract_feature(coughvid_df, 'mfcc', nmfcc=26)
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

2022-03-07 06:37:25,460 - FULL AUDIO 1-D - INFO - 1000 audios extracted !!!
2022-03-07 06:51:43,106 - FULL AUDIO 1-D - INFO - 2000 audios extracted !!!
2022-03-07 07:06:06,729 - FULL AUDIO 1-D - INFO - 3000 audios extracted !!!
2022-03-07 07:20:22,410 - FULL AUDIO 1-D - INFO - 4000 audios extracted !!!
2022-03-07 07:34:49,166 - FULL AUDIO 1-D - INFO - 5000 audios extracted !!!
2022-03-07 07:49:13,590 - FULL AUDIO 1-D - INFO - 6000 audios extracted !!!
2022-03-07 08:03:42,950 - FULL AUDIO 1-D - INFO - 7000 audios extracted !!!
2022-03-07 08:18:12,601 - FULL AUDIO 1-D - INFO - 8000 audios extracted !!!
2022-03-07 08:32:35,580 - FULL AUDIO 1-D - INFO - 9000 audios extracted !!!
2022-03-07 08:47:17,017 - FULL AUDIO 1-D - INFO - 10000 audios extracted !!!
2022-03-07 09:02:13,205 - FULL AUDIO 1-D - INFO - 11000 audios extracted !!!
2022-03-07 09:17:14,645 - FULL AUDIO 1-D - INFO - 12000 audios extracted !!!
2022-03-07 09:32:19,855 - FULL AUDIO 1-D - INFO - 13000 audios extracted !!!
2022-03-

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_26_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_26_coughvid_features.csv'))

2022-03-07 11:17:45,212 - FULL AUDIO 1-D - INFO - /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/ml_model/full_audio/mfcc_26_coughvid_features.csv


In [ ]:
mfcc_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,label
0,-538.235535,26.917747,-3.401038,8.839468,-8.327441,11.548204,-7.401790,-0.828778,-0.154978,-4.501925,...,-2.572593,0.345402,-0.081549,-3.765644,-1.702560,-1.131563,-0.474165,-1.498856,-0.209818,healthy
1,-345.990143,71.909897,-18.871368,12.526669,-17.672796,1.757027,-4.064578,-5.020956,4.243684,-2.339918,...,-5.350095,-4.869955,-4.248755,-3.179543,-3.452577,-3.894665,-0.514441,-1.605611,-0.823914,healthy
2,-1131.371094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3,-433.144104,45.302246,-50.382362,14.131469,-21.137430,13.765966,-25.833303,-8.405109,-22.614456,0.857693,...,-4.494536,0.065545,-2.060712,-3.514311,2.367276,-4.079885,-3.315957,0.591303,0.678734,healthy
4,-336.101135,64.001518,-43.763851,-1.756580,-29.107180,-2.131583,-18.364370,-1.392341,-4.074676,-15.432065,...,-1.451932,1.580137,-0.230461,-8.963498,-0.578151,7.388661,2.276241,-0.233759,6.055211,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20067,-459.546234,34.437195,-28.396717,2.099871,-30.235441,22.874445,-34.096146,9.496261,-11.902689,-1.679841,...,-6.159215,-0.736387,-4.460313,-3.918872,0.717660,-6.636698,-0.609442,-1.053164,1.054812,healthy
20068,-546.165588,11.158798,-19.706007,7.484764,-5.009986,-3.976178,-3.161638,-4.310158,1.712280,0.552225,...,-1.169223,0.097454,0.582215,0.044273,-0.151117,-0.534309,-0.087383,-1.068385,0.731880,symptomatic
20069,-658.566162,21.329943,-14.411014,12.522676,-1.128189,-3.581388,-3.818231,2.658957,-3.306547,-3.568908,...,-1.524021,-0.618467,0.229122,0.161163,-0.778359,-0.857085,-1.272158,-0.081833,0.038856,NaN
20070,-225.830414,145.041504,-47.753567,7.660823,-3.258156,-13.082716,-17.107506,0.686574,-10.240312,-5.894524,...,-5.856904,-2.691986,-5.169966,-4.865459,-2.114848,-6.345662,-1.691320,-2.598970,-3.633305,healthy


In [ ]:
del features
features, labels = extract_feature(coughvid_df, 'mfcc', nmfcc=39)
mfcc_df = pd.DataFrame(features)
mfcc_df['label'] = labels

In [ ]:
mfcc_df.to_csv(os.path.join(root, 'mfcc_39_coughvid_features.csv'), index=0)
logger.info(os.path.join(root, 'mfcc_39_coughvid_features.csv'))